In [1]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
# #     print(dirname)

In [1]:
%cd /kaggle/working
!git clone https://github.com/Harito97/HaritoProduction
%cd HaritoProduction
%ls
!git pull

/kaggle/working
Cloning into 'HaritoProduction'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 128 (delta 40), reused 111 (delta 23), pack-reused 0
Receiving objects: 100% (128/128), 5.15 MiB | 25.72 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/kaggle/working/HaritoProduction
README.md  backend/  docker-compose.yml  frontend/  test/
Already up to date.


In [2]:
import wandb
wandb.login() # 00e43aa49b2b8c17d0db66c858191c6420f4dc9e

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Create data

In [5]:
%pip install ultralytics grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.2-py3-none-any.whl size=38336 sha256=e81507cae3b1e064c85ac30313bb4f49345d9d464fa09fed7ee5f90c212a7c5e
  Stored in directory: /root/.cache/pip/wheels/b4/68/bb/d10381e86dc0de1c9354bce3d86bffcd247305058c40ce2e55
Successfully built grad-cam
Note: you may need to restart the kernel to use updated packages.


In [6]:
from backend.app.models.H97 import H97_ANN
from backend.app.app import App

In [7]:
model1_path = '/kaggle/input/cell-cluster-detection/pytorch/train300epoches/1/best.pt'
model2_path = '/kaggle/input/cellclassifier/pytorch/train100epoches/1/best_h97_retrainEfficientNet_B2_B5_B6_dataver3_model.pt'
app = App(model1_path, model2_path)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 45.8MB/s]


In [8]:
def __np_images_to_tensor_images(np_images):
    tensor_images = torch.from_numpy(np_images)
    # Chuyển đổi thứ tự các trục từ (H, W, C) về (C, H, W)
    tensor_images = tensor_images.permute(0, 3, 1, 2)
    tensor_images = tensor_images.float()
    return tensor_images

In [9]:
import os
import torch

data_dir = '/kaggle/input/thyroidcancer-ver1/dataver1/ver1'
input_info = {}
for dataset in ['train', 'valid', 'test']:
    results = []
    labels = []
    for index_label, label in enumerate(['.B2', 'B5', 'B6']):
        for filename in os.listdir(f'{data_dir}/{dataset}/{label}'):
            if filename.endswith('.jpg'):
                image_path = f'{data_dir}/{dataset}/{label}/{filename}'
            else:
                continue
            print(f'Processing {image_path}')
            images = __np_images_to_tensor_images(app.get_images(image_path))
#             result = []
#             for image in images:
#                 output_predict = app.model2.forward(image)
            result = app.model2.forward(images)
#                 result.append(output_predict)
            results.append(result.tolist())
            labels.append([[index_label] * 18])
    input_info[dataset] = {'results': results, 'labels': labels}

Processing /kaggle/input/thyroidcancer-ver1/dataver1/ver1/train/.B2/B2_44.jpg

image 1/1 /kaggle/input/thyroidcancer-ver1/dataver1/ver1/train/.B2/B2_44.jpg: 480x640 1 cluster, 66.9ms
Speed: 7.5ms preprocess, 66.9ms inference, 78.5ms postprocess per image at shape (1, 3, 480, 640)
/kaggle/input/thyroidcancer-ver1/dataver1/ver1/train/.B2/B2_44.jpg have 1 cell importance clusters
Processing /kaggle/input/thyroidcancer-ver1/dataver1/ver1/train/.B2/B2_41.jpg

image 1/1 /kaggle/input/thyroidcancer-ver1/dataver1/ver1/train/.B2/B2_41.jpg: 480x640 3 clusters, 11.9ms
Speed: 2.5ms preprocess, 11.9ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
/kaggle/input/thyroidcancer-ver1/dataver1/ver1/train/.B2/B2_41.jpg have 3 cell importance clusters
Processing /kaggle/input/thyroidcancer-ver1/dataver1/ver1/train/.B2/B2_4.jpg

image 1/1 /kaggle/input/thyroidcancer-ver1/dataver1/ver1/train/.B2/B2_4.jpg: 480x640 2 clusters, 11.8ms
Speed: 3.2ms preprocess, 11.8ms inference, 0.6ms postproce

In [10]:
# for record in input_info:
#     for result, _ in record:
#         result = result.tolist()

# Save input_info
import json
with open('input_info.json', 'w') as f:
    json.dump(input_info, f)
# Read input_info
with open('input_info.json', 'r') as f:
    input_info = json.load(f)
# Get train dataset
train_dataset = input_info['train']
# Get valid dataset
valid_dataset = input_info['valid']
# Get test dataset
test_dataset = input_info['test']

In [11]:
train_dataset = input_info['train']
# Get valid dataset
valid_dataset = input_info['valid']
# Get test dataset
test_dataset = input_info['test']

In [19]:
print(len(train_dataset['results']), len(train_dataset['results'][0]))

1150 18


In [12]:
import os
import json
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    f1_score,
    roc_curve,
    auc,
)
from sklearn.preprocessing import label_binarize
import wandb

class Tool:
    @staticmethod
    def save_confusion_matrix(y_true, y_score, target_names, filename, normalize=False):
        """
        Saves the confusion matrix to a file.

        Args:
            y_true (array-like): True labels.
            y_score (array-like): Predicted scores.
            target_names (list): Names of the target classes.
            filename (str): Path to save the confusion matrix.
            normalize (bool, optional): Whether to normalize the confusion matrix. Defaults to False.
        Returns:
            cm (numpy.ndarray): The confusion matrix.
        """
        try:
            cm = confusion_matrix(y_true, y_score)
            print("Confusion Matrix:\n", cm)
            if normalize:
                cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
                title = "Normalized Confusion Matrix"
            else:
                title = "Confusion Matrix, Without Normalization"

            plt.figure(figsize=(8, 6))
            sns.heatmap(
                cm,
                annot=True,
                fmt=".2f" if normalize else "d",
                cmap="Blues",
                xticklabels=target_names,
                yticklabels=target_names,
            )
            plt.ylabel("True label")
            plt.xlabel("Predicted label")
            plt.title(title)
            plt.savefig(filename)
            plt.close()
            return cm
        except ValueError as e:
            print(f"Error creating confusion matrix: {e}")
            return None

    @staticmethod
    def save_classification_report(y_true, y_score, filename):
        """
        Saves the classification report to a file.

        Args:
            y_true (array-like): True labels.
            y_score (array-like): Predicted scores.
            filename (str): Path to save the classification report.
        Returns:
            cr (dict): The classification report.
        """
        try:
            cr = classification_report(y_true, y_score, output_dict=True)
            print("Classification Report:\n", cr)
            report_df = pd.DataFrame(cr).transpose()
            report_df.drop(
                "support", axis=1, inplace=True
            )  # Bỏ cột support nếu không cần
            report_df.plot(kind="bar", figsize=(10, 6))
            plt.title("Classification Report")
            plt.ylabel("Score")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(filename)
            plt.close()
            return cr
        except ValueError as e:
            print(f"Error creating DataFrame from classification report: {e}")
            return None

    @staticmethod
    def save_roc_auc_plot(y_true, y_score, n_classes, filename):
        # Chuyển y_true và y_score thành định dạng numpy array
        y_true = np.array(y_true)
        y_score = np.array(y_score)
        
        # Chuyển y_true thành dạng nhị phân
        y_true = label_binarize(y_true, classes=[*range(n_classes)])
        
        # Khởi tạo các biến để lưu kết quả ROC AUC
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        
        # Tính toán ROC AUC cho từng lớp
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        
        # Vẽ ROC curves
        plt.figure()
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve of class {i} (area = {roc_auc[i]:.2f})')
        
        plt.plot([0, 1], [0, 1], 'k--', lw=2)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.savefig(filename)
        plt.close()
#         except ValueError as e:
#             print(f"Error creating DataFrame from classification report: {e}")

In [23]:
x = train_dataset["results"][:32]
y = train_dataset["labels"][:32]

In [30]:
print((train_dataset["results"][0]))

[[4.56027889251709, -0.12482129037380219, -1.0879979133605957], [4.744997024536133, -0.11174905300140381, -1.1251227855682373], [10.558342933654785, 0.29965272545814514, -2.2934954166412354], [0.10638976097106934, -0.023556888103485107, -0.341966450214386], [2.4691109657287598, -0.2554377615451813, -0.6739329099655151], [2.970486640930176, -0.23732849955558777, -0.7684797048568726], [7.872179985046387, 0.10955700278282166, -1.7536275386810303], [6.138964653015137, -0.013100087642669678, -1.4052839279174805], [6.1093244552612305, -0.015197664499282837, -1.3993268013000488], [8.158939361572266, 0.12985053658485413, -1.81126070022583], [12.887232780456543, 0.46446481347084045, -2.7615585327148438], [10.214706420898438, 0.2753341495990753, -2.224431276321411], [24.096776962280273, 1.2577475309371948, -5.014464855194092], [-39.541751861572266, 5.019413471221924, 3.406898021697998], [-39.53479766845703, 5.025789737701416, 3.4112133979797363], [-39.541053771972656, 5.02212381362915, 3.4087460

In [45]:
import torch

# Giả sử train_dataset["results"] và train_dataset["labels"] đã được load từ trước
x = train_dataset["results"][0:32]  # 32 là batch size

# Chuyển đổi x thành tensor và duỗi tensor thành (32, 54)
tensor_x = torch.tensor(x)
flattened_x = tensor_x.view(32, 54)
print(flattened_x.shape)  # Kết quả sẽ là torch.Size([32, 54])
print(flattened_x)

# Giả sử nhãn hiện tại là dạng như [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
y = train_dataset["labels"][0:32]

# Chuyển đổi y thành tensor và đơn giản hóa shape
# Lấy giá trị đầu tiên của mỗi nhãn
simplified_y = [label[0][0] for label in y]
tensor_y = torch.tensor(simplified_y).view(32, 1).view(-1)
print(tensor_y.shape)  # Kết quả sẽ là torch.Size([32, 1])
print(tensor_y)


torch.Size([32, 54])
tensor([[ 4.5603e+00, -1.2482e-01, -1.0880e+00,  ..., -3.9376e+01,  4.9912e+00,  3.3866e+00],
        [ 5.6199e+00, -4.9833e-02, -1.3010e+00,  ..., -3.9343e+01,  5.0000e+00,  3.3924e+00],
        [-2.5190e+01, -4.9874e-01, -4.5021e-01,  ..., -3.9261e+01,  4.9861e+00,  3.3825e+00],
        ...,
        [ 6.8434e+00,  3.6752e-02, -1.5469e+00,  ..., -3.9253e+01,  4.9848e+00,  3.3815e+00],
        [ 7.7895e+00,  1.0370e-01, -1.7370e+00,  ..., -3.9362e+01,  4.9899e+00,  3.3857e+00],
        [ 9.4942e+00,  2.2435e-01, -2.0796e+00,  ..., -3.9279e+01,  4.9834e+00,  3.3807e+00]])
torch.Size([32])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [36]:
y = train_dataset["labels"][0:32]
print(y) # [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]


[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0,

In [28]:
print(len(train_dataset["labels"][0]))

1


In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import f1_score
import json
import wandb

class H97_ANN(nn.Module):
    def __init__(self):
        super(H97_ANN, self).__init__()
        self.fc1 = nn.Linear(12*3 + 5*3 + 1*3, 97)
        self.fc2 = nn.Linear(97, 3)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

    def fix_batch(self, x, y, criterion, optimizer=None):
        x = x.to(device)
        y = y.to(device)
        output = self.forward(x)
        loss = criterion(output, y)
        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        return loss, output

# Số lượng mẫu cho từng lớp
num_samples_B2 = 103
num_samples_B5 = 541
num_samples_B6 = 777

# Tổng số mẫu
total_samples = num_samples_B2 + num_samples_B5 + num_samples_B6
num_classes = 3

# Tính trọng số cho từng lớp
class_weight_B2 = total_samples / (num_classes * num_samples_B2)
class_weight_B5 = total_samples / (num_classes * num_samples_B5)
class_weight_B6 = total_samples / (num_classes * num_samples_B6)

# Tạo tensor chứa trọng số
model = H97_ANN()
class_weights = torch.tensor([class_weight_B2, class_weight_B5, class_weight_B6])
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = criterion.to(device)
model.to(device)

# Def to train the H97_ANN model
def train():
    batch_size = 32
    epochs = 100
    patience = 20
    wandb.init(
        project="ThyroidCancer",
        entity="harito",
        name="H97_ANN_train",
        config={"batch_size": batch_size, "epochs": epochs, "patience": patience},
    )
    step = len(train_dataset["results"]) // batch_size

    history_file_path = f"history.json"
    model_best = f"model_best.pt"
    model_last = f"model_last.pt"

    print("Initializing history ...")
    history = {
        "train_loss": [],
        "train_acc": [],
        "train_f1": [],
        "val_loss": [],
        "val_acc": [],
        "val_f1": [],
    }

    print("Training classification model...")
    best_loss = float("inf")
    patience_counter = 0
    batch_in_current_step = 32
    for i in range(epochs):
        running_loss = 0.0
        train_preds = []
        train_targets = []
        preds = torch.tensor([]).to(device)
        for j in range(step):
            if j * batch_size >= len(train_dataset["results"]):
                batch_in_current_step = len(train_dataset["resultls"]) - i * batch_size
                x = train_dataset["results"][j * batch_size :]
                y = train_dataset["labels"][j * batch_size :]
            else:
                batch_in_current_step = 32
                x = train_dataset["results"][j * batch_size : (j + 1) * batch_size]
                y = train_dataset["labels"][j * batch_size : (j + 1) * batch_size]

            # Chuyển đổi x thành tensor và duỗi tensor thành (32, 54)
            tensor_x = torch.tensor(x)
            x = tensor_x.view(batch_in_current_step, 54)

            simplified_y = [label[0][0] for label in y]
            y = torch.tensor(simplified_y).view(32, 1).view(-1)
            
            loss, output = model.fix_batch(x, y, criterion, optimizer)
            running_loss += loss.item()
            _, batch_preds = torch.max(output, 1)
            train_preds.extend(batch_preds.cpu().numpy())
            train_targets.extend(y.cpu().numpy())

        train_loss = running_loss / len(train_dataset["results"])
        train_acc = np.mean(np.array(train_preds) == np.array(train_targets))
        train_f1 = f1_score(train_targets, train_preds, average="weighted")

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["train_f1"].append(train_f1)
        
        x_valid = valid_dataset["results"]
        tensor_x = torch.tensor(x_valid)
        x_valid = tensor_x.view(len(valid_dataset["results"]), 54)
        
        
        y_valid = valid_dataset["labels"]
        simplified_y = [label[0][0] for label in y_valid]
        y_valid = torch.tensor(simplified_y).view(len(valid_dataset["labels"]), 1).view(-1)
        
        valid_loss, valid_output = model.fix_batch(
            x_valid,
            y_valid,
            criterion
        )
        val_loss = valid_loss.item() / len(valid_dataset["results"])
        _, val_preds = torch.max(valid_output, 1)
        val_acc = np.mean(np.array(val_preds.cpu().numpy()) == np.array(y_valid))
        val_f1 = f1_score(y_valid, val_preds.cpu().numpy(), average="weighted")

        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)
        history["val_f1"].append(val_f1)

        wandb.log(
            {
                "train_loss": train_loss,
                "train_acc": train_acc,
                "train_f1": train_f1,
                "val_loss": val_loss,
                "val_acc": val_acc,
                "val_f1": val_f1,
                "epoch": i,
            }
        )

        with open(history_file_path, "w") as history_file:
            json.dump(history, history_file)

        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), model_best)
            print(f"Model saved with val_loss: {val_loss}")
        else:
            patience_counter += 1
            torch.save(model.state_dict(), model_last)
            print(f"Patience counter: {patience_counter}")

        if patience_counter >= patience:
            print("Early stopping")
            break

    wandb.finish()

def test():
    print("Loading the best model ...")
    model.load_state_dict(torch.load("model_best.pt"))
    model.eval()
    
    x_test = test_dataset["results"]
    tensor_x = torch.tensor(x_test)
    x_test = tensor_x.view(len(test_dataset["results"]), 54)
        
        
    y_test = test_dataset["labels"]
    simplified_y = [label[0][0] for label in y_test]
    y_test = torch.tensor(simplified_y).view(len(test_dataset["labels"]), 1).view(-1)
    
    test_loss, test_output = model.fix_batch(
        x_test,
        y_test,
        criterion
    )
    _, preds = torch.max(test_output, 1)
    test_loss = test_loss.item() / len(test_dataset["results"])
    test_acc = np.mean(np.array(preds.cpu().numpy()) == np.array(y_test))
    test_f1 = f1_score(y_test, preds.cpu().numpy(), average="weighted")
    # Save confusion matrix
    Tool.save_confusion_matrix(
        test_dataset["labels"],
        preds.cpu().numpy(),
        ["B2", "B5", "B6"],
        "confusion_matrix.png",
    )
    # Save classification report
    Tool.save_classification_report(
        test_dataset["labels"],
        preds.cpu().numpy(),
        "classification_report.png",
    )
    # Save ROC AUC plot
    Tool.save_roc_auc_plot(
        test_dataset["labels"], preds.cpu().numpy(), 3, "roc_auc.png"
    )

    print(f"Test loss: {test_loss}, Test accuracy: {test_acc}, Test f1: {test_f1}")
    wandb.init(
        project="ThyroidCancer",
        entity="harito",
        name="H97_ANN_test",
        config={"test_loss": test_loss, "test_acc": test_acc, "test_f1": test_f1},
    )
    wandb.log(
        {
            "test_loss": test_loss,
            "test_acc": test_acc,
            "test_f1": test_f1,
            "confusion_matrix": wandb.Image(f"confusion_matrix.png"),
            "classification_report": wandb.Image(f"classification_report.png"),
            "roc_auc_plot": wandb.Image(f"roc_auc.png"),
        }
    )
    wandb.finish()


In [53]:
train()

Initializing history ...
Training classification model...
Model saved with val_loss: 0.0075442410432375396
Model saved with val_loss: 0.00636337399482727
Model saved with val_loss: 0.0056279012790093055
Patience counter: 1
Model saved with val_loss: 0.005437293419471154
Model saved with val_loss: 0.0054304054150214566
Patience counter: 1
Model saved with val_loss: 0.005282999919011043
Patience counter: 1
Model saved with val_loss: 0.00504684173143827
Patience counter: 1
Patience counter: 2
Patience counter: 3
Patience counter: 4
Patience counter: 5
Patience counter: 6
Patience counter: 7
Model saved with val_loss: 0.004976524756504939
Patience counter: 1
Patience counter: 2
Patience counter: 3
Patience counter: 4
Patience counter: 5
Patience counter: 6
Patience counter: 7
Patience counter: 8
Patience counter: 9
Patience counter: 10
Patience counter: 11
Patience counter: 12
Patience counter: 13
Patience counter: 14
Patience counter: 15
Model saved with val_loss: 0.004975866812926073
Pat

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▅▆▇▇▇██████████████████████████████████
train_f1,▁▄▆▇▇▇██████████████████████████████████
train_loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▄▆▇▇▇▇█▇▇▇▇██▇█▇▇███▇██▇██▇█▇▇▇▇▇▇█▇▇▇▇
val_f1,▁▄▆▇▇▇▇█▇▇▇▇██▇█▇▇███▇██▇██▇█▇▇▇▇▇▇█▇▇▇▇
val_loss,█▅▃▂▂▃▂▁▁▂▁▁▁▁▂▁▂▁▁▁▁▂▂▁▁▁▁▂▁▁▁▁▁▁▂▁▂▁▂▂
epoch,53
train_acc,0.97321
train_f1,0.97323
train_loss,0.01761


In [62]:
import torch
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import wandb

# Giả sử Tool.save_confusion_matrix, Tool.save_classification_report, và Tool.save_roc_auc_plot đã được định nghĩa

def test():
    print("Loading the best model ...")
    model.load_state_dict(torch.load("model_best.pt"))
    model.eval()
    
    x_test = test_dataset["results"]
    tensor_x = torch.tensor(x_test)
    x_test = tensor_x.view(len(test_dataset["results"]), 54)
        
    y_test = test_dataset["labels"]
    simplified_y = [label[0][0] for label in y_test]  # Chỉ lấy giá trị đầu tiên của mỗi nhãn
    y_test = torch.tensor(simplified_y).view(-1)  # Chuyển thành tensor 1D
    
    test_loss, test_output = model.fix_batch(
        x_test,
        y_test,
        criterion
    )
    _, preds = torch.max(test_output, 1)
    test_loss = test_loss.item() / len(test_dataset["results"])
    test_acc = np.mean(preds.cpu().numpy() == y_test.numpy())
    test_f1 = f1_score(y_test, preds.cpu().numpy(), average="weighted")
    
    print('y_test = ', y_test)
    print('y_test.shape = ', y_test.shape)
    print('preds = ', preds)
    print('preds.shape = ', preds.shape)
    
    # Save confusion matrix
    Tool.save_confusion_matrix(
        y_test.numpy(),
        preds.cpu().numpy(),
        ["B2", "B5", "B6"],
        "confusion_matrix.png",
    )
    # Save classification report
    Tool.save_classification_report(
        y_test.numpy(),
        preds.cpu().numpy(),
        "classification_report.png",
    )
    # Save ROC AUC plot
    Tool.save_roc_auc_plot(
        y_test.numpy(), preds.cpu().numpy(), 3, "roc_auc.png"
    )

    print(f"Test loss: {test_loss}, Test accuracy: {test_acc}, Test f1: {test_f1}")
    wandb.init(
        project="ThyroidCancer",
        entity="harito",
        name="H97_ANN_test",
        config={"test_loss": test_loss, "test_acc": test_acc, "test_f1": test_f1},
    )
    wandb.log(
        {
            "test_loss": test_loss,
            "test_acc": test_acc,
            "test_f1": test_f1,
            "confusion_matrix": wandb.Image("confusion_matrix.png") if os.path.exists("confusion_matrix.png") else None,
            "classification_report": wandb.Image("classification_report.png") if os.path.exists("classification_report.png") else None,
            "roc_auc_plot": wandb.Image("roc_auc.png") if os.path.exists("roc_auc.png") else None,
        }
    )
    wandb.finish()

# Đảm bảo rằng các hàm Tool.save_confusion_matrix, Tool.save_classification_report, và Tool.save_roc_auc_plot đã được định nghĩa trước đó và lưu tệp đúng cách.


In [66]:
def test():
    print("Loading the best model ...")
    model.load_state_dict(torch.load("model_best.pt"))
    model.eval()
    
    x_test = test_dataset["results"]
    tensor_x = torch.tensor(x_test, dtype=torch.float32)
    x_test = tensor_x.view(len(test_dataset["results"]), 54)
        
    y_test = test_dataset["labels"]
    simplified_y = [label[0][0] for label in y_test]
    y_test = torch.tensor(simplified_y, dtype=torch.long)

    # Chuyển dữ liệu vào thiết bị (CPU hoặc GPU)
    x_test = x_test.to(device)
    y_test = y_test.to(device)

    # Tiến hành dự đoán
    with torch.no_grad():
        test_output = model(x_test)
        test_loss = criterion(test_output, y_test)
        _, preds = torch.max(test_output, 1)
    
    test_loss = test_loss.item() / len(test_dataset["results"])
    test_acc = np.mean(preds.cpu().numpy() == y_test.cpu().numpy())
    test_f1 = f1_score(y_test.cpu().numpy(), preds.cpu().numpy(), average="weighted")

    # Save confusion matrix
    Tool.save_confusion_matrix(
        y_test.cpu().numpy(),
        preds.cpu().numpy(),
        ["B2", "B5", "B6"],
        "confusion_matrix.png",
    )
    # Save classification report
    Tool.save_classification_report(
        y_test.cpu().numpy(),
        preds.cpu().numpy(),
        "classification_report.png",
    )
    # Save ROC AUC plot
    y_score = torch.softmax(test_output, dim=1).cpu().numpy()  # Chuyển đổi đầu ra của mô hình thành xác suất
    Tool.save_roc_auc_plot(
        y_test.cpu().numpy(), y_score, 3, "roc_auc.png"
    )

    print(f"Test loss: {test_loss}, Test accuracy: {test_acc}, Test f1: {test_f1}")
    wandb.init(
        project="ThyroidCancer",
        entity="harito",
        name="H97_ANN_test",
        config={"test_loss": test_loss, "test_acc": test_acc, "test_f1": test_f1},
    )
    wandb.log(
        {
            "test_loss": test_loss,
            "test_acc": test_acc,
            "test_f1": test_f1,
            "confusion_matrix": wandb.Image("confusion_matrix.png") if os.path.exists("confusion_matrix.png") else None,
            "classification_report": wandb.Image("classification_report.png") if os.path.exists("classification_report.png") else None,
            "roc_auc_plot": wandb.Image("roc_auc.png") if os.path.exists("roc_auc.png") else None,
        }
    )
    wandb.finish()


In [67]:
test()

Loading the best model ...
Confusion Matrix:
 [[ 6  5  0]
 [ 0 50  5]
 [ 0  8 71]]
Classification Report:
 {'0': {'precision': 1.0, 'recall': 0.5454545454545454, 'f1-score': 0.7058823529411764, 'support': 11}, '1': {'precision': 0.7936507936507936, 'recall': 0.9090909090909091, 'f1-score': 0.847457627118644, 'support': 55}, '2': {'precision': 0.9342105263157895, 'recall': 0.8987341772151899, 'f1-score': 0.9161290322580645, 'support': 79}, 'accuracy': 0.8758620689655172, 'macro avg': {'precision': 0.9092871066555276, 'recall': 0.7844265439202148, 'f1-score': 0.8231563374392951, 'support': 145}, 'weighted avg': {'precision': 0.8858856912395932, 'recall': 0.8758620689655172, 'f1-score': 0.8741315098087272, 'support': 145}}
Test loss: 0.0052825323466596935, Test accuracy: 0.8758620689655172, Test f1: 0.8741315098087272


test_acc,▁
test_f1,▁
test_loss,▁
test_acc,0.87586
test_f1,0.87413
test_loss,0.00528


In [69]:
def test():
    # Test cho tập train xem nó dự đoán ra sao
    print("Loading the best model ...")
    model.load_state_dict(torch.load("model_best.pt"))
    model.eval()
    
    x_test = train_dataset["results"]
    tensor_x = torch.tensor(x_test, dtype=torch.float32)
    x_test = tensor_x.view(len(train_dataset["results"]), 54)
        
    y_test = train_dataset["labels"]
    simplified_y = [label[0][0] for label in y_test]
    y_test = torch.tensor(simplified_y, dtype=torch.long)

    # Chuyển dữ liệu vào thiết bị (CPU hoặc GPU)
    x_test = x_test.to(device)
    y_test = y_test.to(device)

    # Tiến hành dự đoán
    with torch.no_grad():
        test_output = model(x_test)
        test_loss = criterion(test_output, y_test)
        _, preds = torch.max(test_output, 1)
    
    test_loss = test_loss.item() / len(train_dataset["results"])
    test_acc = np.mean(preds.cpu().numpy() == y_test.cpu().numpy())
    test_f1 = f1_score(y_test.cpu().numpy(), preds.cpu().numpy(), average="weighted")

    # Save confusion matrix
    Tool.save_confusion_matrix(
        y_test.cpu().numpy(),
        preds.cpu().numpy(),
        ["B2", "B5", "B6"],
        "confusion_matrix.png",
    )
    # Save classification report
    Tool.save_classification_report(
        y_test.cpu().numpy(),
        preds.cpu().numpy(),
        "classification_report.png",
    )
    # Save ROC AUC plot
    y_score = torch.softmax(test_output, dim=1).cpu().numpy()  # Chuyển đổi đầu ra của mô hình thành xác suất
    Tool.save_roc_auc_plot(
        y_test.cpu().numpy(), y_score, 3, "roc_auc.png"
    )

    print(f"Test loss: {test_loss}, Test accuracy: {test_acc}, Test f1: {test_f1}")
    wandb.init(
        project="ThyroidCancer",
        entity="harito",
        name="H97_ANN_test_in_trainset",
        config={"test_loss": test_loss, "test_acc": test_acc, "test_f1": test_f1},
    )
    wandb.log(
        {
            "test_loss": test_loss,
            "test_acc": test_acc,
            "test_f1": test_f1,
            "confusion_matrix": wandb.Image("confusion_matrix.png") if os.path.exists("confusion_matrix.png") else None,
            "classification_report": wandb.Image("classification_report.png") if os.path.exists("classification_report.png") else None,
            "roc_auc_plot": wandb.Image("roc_auc.png") if os.path.exists("roc_auc.png") else None,
        }
    )
    wandb.finish()
test()

Loading the best model ...
Confusion Matrix:
 [[ 77   5   0]
 [  1 433   3]
 [  0  23 608]]
Classification Report:
 {'0': {'precision': 0.9871794871794872, 'recall': 0.9390243902439024, 'f1-score': 0.9625, 'support': 82}, '1': {'precision': 0.9392624728850325, 'recall': 0.9908466819221968, 'f1-score': 0.9643652561247216, 'support': 437}, '2': {'precision': 0.9950900163666121, 'recall': 0.9635499207606973, 'f1-score': 0.9790660225442835, 'support': 631}, 'accuracy': 0.9721739130434782, 'macro avg': {'precision': 0.9738439921437106, 'recall': 0.9644736643089322, 'f1-score': 0.968643759556335, 'support': 1150}, 'weighted avg': {'precision': 0.9733114947189646, 'recall': 0.9721739130434782, 'f1-score': 0.9722985018712575, 'support': 1150}}
Test loss: 0.0005071902275085449, Test accuracy: 0.9721739130434782, Test f1: 0.9722985018712575


test_acc,▁
test_f1,▁
test_loss,▁
test_acc,0.97217
test_f1,0.9723
test_loss,0.00051


In [70]:
def test():
    # Test cho tập train xem nó dự đoán ra sao
    print("Loading the best model ...")
    model.load_state_dict(torch.load("model_best.pt"))
    model.eval()
    
    x_test = valid_dataset["results"]
    tensor_x = torch.tensor(x_test, dtype=torch.float32)
    x_test = tensor_x.view(len(valid_dataset["results"]), 54)
        
    y_test = valid_dataset["labels"]
    simplified_y = [label[0][0] for label in y_test]
    y_test = torch.tensor(simplified_y, dtype=torch.long)

    # Chuyển dữ liệu vào thiết bị (CPU hoặc GPU)
    x_test = x_test.to(device)
    y_test = y_test.to(device)

    # Tiến hành dự đoán
    with torch.no_grad():
        test_output = model(x_test)
        test_loss = criterion(test_output, y_test)
        _, preds = torch.max(test_output, 1)
    
    test_loss = test_loss.item() / len(valid_dataset["results"])
    test_acc = np.mean(preds.cpu().numpy() == y_test.cpu().numpy())
    test_f1 = f1_score(y_test.cpu().numpy(), preds.cpu().numpy(), average="weighted")

    # Save confusion matrix
    Tool.save_confusion_matrix(
        y_test.cpu().numpy(),
        preds.cpu().numpy(),
        ["B2", "B5", "B6"],
        "confusion_matrix.png",
    )
    # Save classification report
    Tool.save_classification_report(
        y_test.cpu().numpy(),
        preds.cpu().numpy(),
        "classification_report.png",
    )
    # Save ROC AUC plot
    y_score = torch.softmax(test_output, dim=1).cpu().numpy()  # Chuyển đổi đầu ra của mô hình thành xác suất
    Tool.save_roc_auc_plot(
        y_test.cpu().numpy(), y_score, 3, "roc_auc.png"
    )

    print(f"Test loss: {test_loss}, Test accuracy: {test_acc}, Test f1: {test_f1}")
    wandb.init(
        project="ThyroidCancer",
        entity="harito",
        name="H97_ANN_test_in_validset",
        config={"test_loss": test_loss, "test_acc": test_acc, "test_f1": test_f1},
    )
    wandb.log(
        {
            "test_loss": test_loss,
            "test_acc": test_acc,
            "test_f1": test_f1,
            "confusion_matrix": wandb.Image("confusion_matrix.png") if os.path.exists("confusion_matrix.png") else None,
            "classification_report": wandb.Image("classification_report.png") if os.path.exists("classification_report.png") else None,
            "roc_auc_plot": wandb.Image("roc_auc.png") if os.path.exists("roc_auc.png") else None,
        }
    )
    wandb.finish()
test()

Loading the best model ...
Confusion Matrix:
 [[ 9  1  0]
 [ 0 43  6]
 [ 0  6 65]]
Classification Report:
 {'0': {'precision': 1.0, 'recall': 0.9, 'f1-score': 0.9473684210526316, 'support': 10}, '1': {'precision': 0.86, 'recall': 0.8775510204081632, 'f1-score': 0.8686868686868686, 'support': 49}, '2': {'precision': 0.9154929577464789, 'recall': 0.9154929577464789, 'f1-score': 0.9154929577464789, 'support': 71}, 'accuracy': 0.9, 'macro avg': {'precision': 0.9251643192488263, 'recall': 0.8976813260515474, 'f1-score': 0.9105160824953265, 'support': 130}, 'weighted avg': {'precision': 0.9010769230769231, 'recall': 0.9, 'f1-score': 0.900302621355253, 'support': 130}}
Test loss: 0.005031892886528602, Test accuracy: 0.9, Test f1: 0.900302621355253


test_acc,▁
test_f1,▁
test_loss,▁
test_acc,0.9
test_f1,0.9003
test_loss,0.00503
